# A playground for Amazon Rekognition


Make sure your default aws credentials are properly defined. In linux, the credentials file lives in `~/.aws/credentials`.

In [3]:
s3_bucket = 'datarock-textract-test'
input_dir = 'data'
results_dir = 'rekognition-results'

In [4]:
import boto3
import cv2
import numpy as np

In [5]:
#get list of documents in the s3 bucket/input_dir
import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(s3_bucket)
img_names=[]
for object_summary in my_bucket.objects.filter(Prefix=input_dir+'/'):
    
    file_name=object_summary.key.replace(input_dir+'/','')
    if len(file_name)>0 :
        img_names.append(file_name )

print(f'{len(img_names)} files found in {"s3://"+s3_bucket+"/"+input_dir} directory')

7 files found in s3://datarock-textract-test/data directory


In [16]:
def detect_text(photo, bucket):

    client=boto3.client('rekognition')

    response=client.detect_text(Image={'S3Object':{'Bucket':bucket,'Name':photo}})
                        
    textDetections=response['TextDetections']
#     print ('Detected text\n----------')
#     for text in textDetections:
#             print ('Detected text:' + text['DetectedText'])
#             print ('Confidence: ' + "{:.2f}".format(text['Confidence']) + "%")
#             print ('Id: {}'.format(text['Id']))
#             if 'ParentId' in text:
#                 print ('Parent Id: {}'.format(text['ParentId']))
#             print ('Type:' + text['Type'])
#             print()
    return response['TextDetections']

In [17]:
responses={}
for img_name in img_names:
    print(img_name)
    response=detect_text(input_dir+'/'+img_name , s3_bucket)
    responses[img_name]=response

CE002_142+143_469.20_476.20.jpg
UE108_005+006_13.90_20.95.JPG
UE108_013+014_42.15_48.85.JPG
UE108_021+022_69.25_76.05.JPG
UE108_029+030_97.15_104.15.JPG
UE108_063+064_217.25_224.00.JPG
UE108_089+090_308.00_314.95.JPG


In [18]:
#Let's draw the found text on the image and save the image to the output directory

In [19]:
responses['CE002_142+143_469.20_476.20.jpg']

[{'DetectedText': '5032 TRAY 142',
  'Type': 'LINE',
  'Id': 0,
  'Confidence': 82.93951416015625,
  'Geometry': {'BoundingBox': {'Width': 0.20028550922870636,
    'Height': 0.02435418777167797,
    'Left': 0.06528335809707642,
    'Top': 0.07405140995979309},
   'Polygon': [{'X': 0.06528335809707642, 'Y': 0.07405140995979309},
    {'X': 0.26556888222694397, 'Y': 0.07542625069618225},
    {'X': 0.2654940187931061, 'Y': 0.09978044033050537},
    {'X': 0.06520850211381912, 'Y': 0.09840559214353561}]}},
 {'DetectedText': 'ILT',
  'Type': 'LINE',
  'Id': 1,
  'Confidence': 90.89039611816406,
  'Geometry': {'BoundingBox': {'Width': 0.0,
    'Height': 0.0,
    'Left': 0.8837209343910217,
    'Top': 0.20739910006523132},
   'Polygon': [{'X': 0.8837209343910217, 'Y': 0.20739910006523132},
    {'X': 0.8837209343910217, 'Y': 0.2556053698062897},
    {'X': 0.8582145571708679, 'Y': 0.2556053698062897},
    {'X': 0.8582145571708679, 'Y': 0.20739910006523132}]}},
 {'DetectedText': '72',
  'Type': 'L

In [20]:
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
def download_into_memory(s3_client,bucket_name,file_url):
    s3_response_object = s3_client.get_object(Bucket=bucket_name, Key=file_url)
    object_content = s3_response_object['Body'].read()
    return object_content
def convert_to_img(img_content):
    image = np.asarray(bytearray(img_content), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

def save_img_to_s3_bucket(img,s3_client,bucket_name,file_url):
    img_str = cv2.imencode('.jpg', img)[1].tostring()
    s3_client.put_object(Bucket=bucket_name, Key = file_url, Body = img_str, ContentType= 'image/jpeg')  

In [23]:
def get_color_codes():
#     colors={} #Colors in BGR
#     colors['PAGE'] = (128,0,0) 
#     colors['LINE']= (128,0,0) 
#     colors['WORD'] = (0,128,0) 
#     return colors
    colors={} #Colors in BGR\n",
    colors['PAGE'] = (255,0,0) # Blue
    colors['WORD']= (0,255,0) #Green
    colors['LINE'] = (0,0,255) #Red
    return colors


In [54]:
font = cv2.FONT_HERSHEY_SIMPLEX

def convert_polygon_to_coords(img_size,polygon):
    #get bbox from polygon
    coord= polygon[0]
    min_x = coord['X']
    min_y = coord['Y']
    max_x = coord['X']
    max_y = coord['Y']
    
    
    for coord in polygon:
        x = coord['X']
        y = coord['Y']
        
        if x< min_x:
            min_x=x
        if x>max_x:
            max_x = x
        if y<min_y:
            min_y = y
        if y>max_y:
            max_y = y
    bbox={'Left':min_x,'Top': min_y , 'Width':max_x-min_x , 'Height':max_y-min_y}
            
        
    #x coords
    start_point_x = int(img_size[1]* bbox['Left'])
    end_point_x = int(start_point_x + img_size[1]*bbox['Width'])
    #y coords
    start_point_y = int(img_size[0] * bbox['Top'])
    end_point_y = int(start_point_y + img_size[0] * bbox['Height'])
    
    return (start_point_x,start_point_y) , (end_point_x,end_point_y)

# def draw_on_top_of_img(img,textract_results,color_codes):
#     for text_result in textract_results:
#         if 'Text' in text_result.keys() and text_result['BlockType'] == 'LINE' :
#             drawing_color = color_codes[text_result['BlockType']]
#             start_pnt,end_pnt = convert_bbox_to_coords(img.shape,text_result['Geometry']['BoundingBox'])
#             img = cv2.rectangle(img, start_pnt, end_pnt, (128,30,0), thickness=1) 
# #             print(text_result['BlockType'])
# #             print(text_result['Text'])
# #             print('    \n')
#             cv2.putText(img, text_result['Text'], (end_pnt[0]+2,start_pnt[1]+5), font, 2, (0,0,128), 1, cv2.LINE_AA)
#     return img

def draw_on_top_of_img(img,textract_results,color_codes):
    for text_result in textract_results:
        if text_result['Type'] == 'WORD':
            drawing_color = color_codes[text_result['Type']]
            start_pnt,end_pnt = convert_polygon_to_coords(img.shape,text_result['Geometry']['Polygon'])
            img = cv2.rectangle(img, start_pnt, end_pnt, drawing_color, thickness=2) 
            cv2.putText(img, text_result['DetectedText'], (start_pnt[0]+10,start_pnt[1]+30), font, 1, drawing_color, 2, cv2.LINE_AA)
        
    return img

In [55]:
color_codes = get_color_codes()
for img_name in img_names:
    print(img_name)
    img_content = download_into_memory(s3_client,s3_bucket,input_dir+'/'+img_name)
    img = convert_to_img(img_content)
    resp = responses[img_name]
    img=draw_on_top_of_img(img,resp,color_codes)
    save_img_to_s3_bucket(img,s3_client,s3_bucket,results_dir+'/'+img_name)

CE002_142+143_469.20_476.20.jpg
UE108_005+006_13.90_20.95.JPG
UE108_013+014_42.15_48.85.JPG
UE108_021+022_69.25_76.05.JPG
UE108_029+030_97.15_104.15.JPG
UE108_063+064_217.25_224.00.JPG
UE108_089+090_308.00_314.95.JPG
